In [194]:
Min_Int = -2**31
Max_Int = 2**31-1

import ast, copy

from enum import Enum
class Translate_Actions (Enum):
    PROC_DEC      = 1
    FUNC_DEC      = 2
    FUNC_RESULT   = 3
    IN_PARAM      = 4
    OUT_PARAM     = 5
    IN_OUT_PARAM  = 6
    IN_GLOBAL     = 7
    IN_OUT_GLOBAL = 8
    LOCAL_RW      = 9
    LOCAL_RO      = 10
    EVAL          = 11
    ASSIGN        = 12
    DEFINE_CONST  = 13
    DEFER_CONST   = 14
    IF_COND       = 15
    ELSE_COND     = 16
    ELSE_BRANCH   = 17
    END_IF        = 18
    @classmethod
    def pos (cls, act):
        return act.value - 1
        
Action = Enum('Action', ['PROC_DEC', 'FUNC_DEC', 'FUNC_RESULT', 
                         'IN_PARAM', 'OUT_PARAM', 'IN_OUT_PARAM',
                         'IN_GLOBAL', 'OUT_GLOBAL', 'IN_OUT_GLOBAL', 
                         'LOCAL_RW', 'LOCAL_RO',
                         'EVAL', 'ASSIGN', 'DEFINE_CONST', 'DEFER_CONST', 
                         'IF_COND', 'ELSE_COND', 'ELSE_BRANCH', 'END_IF'])

Mode = Enum('Mode', ['IN_PARAM', 'OUT_PARAM', 'IN_OUT_PARAM', 'IN_GLOBAL', 'IN_OUT_GLOBAL', 'LOCAL_RW', 'LOCAL_RO'])

vars        = {}
var_count   = 0
state_stack = [] # [Var_State, ...]
frame_ptr   = 0
frame       = 0

# traversal_condition (True/False, Condition)
traversal_condition = []
branch_stack = [] # [(action, traversal_condition, (last_frame_ptr, last_frame)), ...]
top         = -1
action      = 0
tcond       = 1
frame_param = 2
ptr         = 0
size        = 1


class Var_State:
    def __init__(self, name, mode, well_formed, scalar, lb, ub, invariant):
        self.name        = name
        self.mode        = mode
        self.well_formed = well_formed
        self.scalar      = scalar
        self.lb          = lb
        self.ub          = ub
        self.invariant   = invariant
        self.evaluated   = False
        self.assigned    = False
        self.updated     = False
        
    def print_var_state(self):
        print('name =', self.name)
        print('mode =', self.mode)
        print('well_formed =', self.well_formed)
        print('scalar =', self.scalar)
        print('lb =', self.lb)
        print('ub =', self.ub)
        print('invariant =', self.invariant)
        print('evaluated =', self.evaluated)
        print('assigned =', self.assigned)
        print('updated =', self.updated)
            
def print_state_stack(stack):
    for i in range(len(stack)):
        stack[i].print_var_state()
        print('----------------------')

def new_frame(from_frame):
    result = []
    for i in range (from_frame[ptr], from_frame[ptr] + from_frame[size]):
        result = result + [copy.deepcopy(state_stack[i])]
        result[-1].updated = False
    return result

def merge(frame_1, frame_2):
    print("merging ", frame_1, " and ", frame_2)



In [195]:
print_state_stack(state_stack)

In [199]:
# Each statement is a list of actions on variables
def process_statement (actions):
    global vars, var_count, state_stack, frame_ptr, frame, branch_stack, branch_ptr, traversal_condition
    for act in actions:
        if act[0] == Action.PROC_DEC:
            frame_ptr = frame_ptr + frame
        elif act[0] == Action.IN_PARAM:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.IN_PARAM, 
                                                   well_formed=(True, []),
                                                   scalar=True, 
                                                   lb=Min_Int, ub=Max_Int,
                                                   invariant=True)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.OUT_PARAM:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.OUT_PARAM, 
                                                   well_formed=(False, []), 
                                                   scalar=True,
                                                   lb=None, ub=None,
                                                   invariant=False)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.IN_OUT_PARAM:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.IN_OUT_PARAM, 
                                                   well_formed=(True, []), 
                                                   scalar=True, 
                                                   lb=Min_Int, ub=Max_Int,
                                                   invariant=False)]
            frame = frame + 1        
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.IN_GLOBAL:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.IN_GLOBAL, 
                                                   well_formed=(True, []),
                                                   scalar=True, 
                                                   lb=Min_Int, ub=Max_Int,
                                                   invariant=True)]
        elif act[0] == Action.OUT_GLOBAL:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.OUT_GLOBAL, 
                                                   well_formed=(False, []), 
                                                   scalar=True, 
                                                   lb=None, ub=None,
                                                   invariant=False)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.IN_OUT_GLOBAL:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.IN_OUT_GLOBAL, 
                                                   well_formed=(True, []),
                                                   scalar=True, 
                                                   lb=Min_Int, ub=Max_Int,
                                                   invariant=False)]
            frame = frame + 1        
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.LOCAL_RW:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.LOCAL_RW, 
                                                   well_formed=(False, []), 
                                                   scalar=True, 
                                                   lb=None, ub=None,
                                                   invariant=False)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.LOCAL_RO:
            state_stack = state_stack + [Var_State(name=act[1], 
                                                   mode=Mode.LOCAL_RO, 
                                                   well_formed=(False, []), 
                                                   scalar=True, 
                                                   lb=None, ub=None,
                                                   invariant=False)]
            frame = frame + 1
            vars[act[1]] = var_count
            var_count = var_count + 1
        elif act[0] == Action.EVAL:
            state_stack[vars[act[1]] + frame_ptr].evaluated=True
        elif act[0] == Action.ASSIGN:
            state_index = vars[act[1]] + frame_ptr
            if state_stack[state_index].assigned:
                state_stack[state_index].updated = True
            else:
                state_stack[state_index].assigned = True

            if state_stack[state_index].well_formed[0] == False:
                # Add the traversal condition on which this variable is well-formed
                state_stack[state_index].well_formed = (True, copy.deepcopy(traversal_condition))
        elif act[0] == Action.IF_COND:
            # The start of a compound statement.
            # Save the current traversal condition and state_stack parameters onto the branch_stack.
            branch_stack = branch_stack + [(Action.IF_COND, 
                                            copy.deepcopy(traversal_condition), (frame_ptr, frame))]
            # Start a new stack frame for the branch. It is the same as the frame prior to the IF_COND
            state_stack = state_stack + new_frame((frame_ptr, frame))
            frame_ptr = frame_ptr + frame
            # Add the condition of the IF_COND to the traversal condition
            # The current traversal condition is extended to include the IF_COND
            if traversal_condition == []:
                traversal_condition =  [(True, ast.dump(ast.parse(act[1], mode='eval')))]
            else:
                traversal_condition = traversal_condition + [(True, 
                                                              ast.dump(ast.parse(act[1], mode='eval')))]

            # May be able to adjust bounds of variables used in condition.
            # for instance, if condition is 'x > y + 10' then x lower bound is lb(y) + 10
        elif act[0] == Action.ELSE_COND:
            # Check whether the previous branch was the IF branch of the IF statement.
            if branch_stack[top][action] == Action.IF_COND:
                # Then save the traversal condition and state_stack frame parameters on the branch_stack
                branch_stack = branch_stack + [(Action.ELSE_COND,
                                                traversal_condition, (frame_ptr, frame))]
            else:
                # It is not an IF branch, merge the last two branches.
                # The resulting merged state is in the lower of the two state_stack frames.
                merge((branch_stack[top-1][frame_param]), (branch_stack[top][frame_param]))
                # Pop off the top state_stack frame.
                state_stack = state_stack[range(0, frame_ptr)]
                frame_ptr = frame_ptr - frame

            # Start a new state_stack frame from the IF_COND frame.
            # The IF_COND frame parameters are always available from the penultimate branch_stack
            # entry as all ELSE_COND branches are merged and have the same branch_stack entry
            state_stack = state_stack + new_frame(branch_stack[top-1][frame_param])
            frame_ptr = frame_ptr + frame
            
            # The traversal for this branch is the same as the previous IF_COND or ELSE_COND branch
            # but with the IF_COND or ELSE_COND negated and traversal condition extend 
            # with the condition of this ELSE_COND.

            # Set the last condition in the traversal condition FALSE
            print("traversal before", traversal_condition)
            print("previous", traversal_condition[-1])
            print("prev expr", traversal_condition[-1][1])
            traversal_condition [-1] = (False, traversal_condition[-1][1])
            print("after", traversal_condition)
            # Now extend the traversal condition with the ELSE_COND.
            traversal_condition = traversal_condition + [(True, 
                                                          ast.dump(ast.parse(act[1], mode='eval')))]
        elif act[0] == Action.ELSE_BRANCH:
            # Check whether the previous branch was the IF branch of the IF statement.
            if branch_stack[top][action] != Action.IF_COND:
                # It is not an IF branch, merge the last two branches
                # The resulting merged state is in the lower of the two state_stack frames.
                merge((branch_stack[top-1][frame_param]), (branch_stack[top][frame_param]))
                # Pop off the top state_stack frame.
                frame_ptr = frame_ptr - frame
                state_stack = state_stack[:frame_ptr]
                # Pop off the ELSE_COND from the branch_stack
                branch_stack = branch_stack[:len(branch_stack)-1]
            
            # Save the traversal condition and current stack frame parameters on the branch_stack
            # and start a new stack frame for the ELSE branch.
            branch_stack = branch_stack + [(Action.ELSE_BRANCH,
                                            traversal_condition, (frame_ptr, frame))]
            state_stack = state_stack + new_frame((frame_ptr, frame))
            frame_ptr = frame_ptr + frame
            # Set the last condition in the traversal condition FALSE
            traversal_condition [-1] = (False, traversal_condition[-1][1])
            # The traversal_condition is not extended for an ELSE branch
        elif act[0] == Action.END_IF:
            # Check whether there is not an ELSE branch
            if branch-stack[top][action] != Action.ELSE_BRANCH:
                # There is no ELSE branch. the merged branches of the IF statement
                # have to be merged with the state preceding the IF_COND
                
                # Check whether the previous branch is not an IF branch of the IF statement.
                if branch_stack[top][action] != Action.IF_COND:
                    # It is not an IF branch, merge the last two branches
                    # The resulting merged state is in the lower of the two state_stack frames.
                    # That is the IF_COND state frame
                    merge((branch_stack[top-1][frame_param]), (branch_stack[top][frame_param]))
                    # Pop off the top state_stack frame.
                    frame_ptr = frame_ptr - frame
                    state_stack = state_stack[:frame_ptr]
                    # Pop off the ELSE_COND from the branch_stack
                    branch_stack = branch_stack[:len(branch_stack)-1]
                    
                # Now merge the IF_COND state frame with the frame preceding the IF_COND  
                # The resulting merged state is in the state frame preceding the IF_COND.
                merge((branch_stack[top-1][frame_param]), (branch_stack[top][frame_param]))
                # Pop off the top state_stack frame.
                frame_ptr = frame_ptr - frame
                state_stack = state_stack[:frame_ptr] 
                # Pop off the IF_COND from the branch_stack
                branch_stack = branch_stack[:len(branch_stack)-1]
            else:
                # There is an ELSE branch.  The merged branches of the IF-THEN-ELSE statement
                # is the state that is carried forward in the analysis.
                # The state prior to the IF_COND has been accounted for in each branch
                # of the IF_THEN_ELSE and is no longer required.

                # Merge the state ELSE_BRANCH and the previous accumulated states in the IF_COND branch.
                # The result of the merge is in the state of the IF_COND branch.
                merge((branch_stack[top-1][frame_param]), (branch_stack[top][frame_param]))
                # Pop off the top state_stack frame.
                frame_ptr = frame_ptr - frame
                state_stack = state_stack[:frame_ptr]
                # Pop off the ELSE_BRANCH from the branch_stack
                branch_stack = branch_stack[:len(branch_stack)-1]

                # Replace the state prior to the IF_COND with the state from the IF_COND branch and
                # pop off IF_COND branch state
                (pre_ptr, pre_frame) = branch_stack[top][frame_params]
                state_stack = state_stack[:pre_ptr] + state_stack[frame_ptr:frame_ptr+frame]
                frame_ptr = pre_ptr  # The IF statement may have introduced new variables,
                                     # although no longer in scope, they have been added to
                                     # the abstract state and therefore the frame count is unchanged.

                # Pop off IF_COND from the branch_stack.
                branch_stack = branch_stack[:len(branch_stack)-1]
        else:
            print("Unsupported ", act[0]) 
                    

In [200]:
process_statement([(Action.PROC_DEC, 'p'), 
                   (Action.IN_OUT_PARAM, 'x'),
                  (Action.IN_OUT_PARAM, 'y')])
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = False
assigned = False
updated = False
----------------------


In [201]:
vars

{'x': 0, 'y': 1}

In [202]:
process_statement([(Action.LOCAL_RW, 'temp')])
print_state_stack(state_stack)               

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = False
updated = False
----------------------


In [203]:
process_statement([(Action.EVAL, 'x'),
                  (Action.EVAL, 'y'),
                  (Action.IF_COND, 'x > y + 10')])
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOC

In [204]:
branch_stack

[(<Action.IF_COND: 16>, [], (0, 3))]

In [205]:
traversal_condition

[(True,
  "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))")]

In [206]:
vars

{'x': 0, 'y': 1, 'temp': 2}

In [207]:
process_statement([(Action.EVAL,'x'), (Action.ASSIGN, 'temp')])

In [208]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOC

In [209]:
process_statement([(Action.EVAL, 'y'), (Action.ASSIGN, 'x')])

In [210]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = True
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCA

In [211]:
process_statement([(Action.EVAL,'temp'), (Action.ASSIGN, 'y')])

In [212]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = True
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = True
updated = False
----------------------
name = temp
mode = Mode.LOCAL

In [213]:
process_statement([(Action.EVAL, 'x'), (Action.EVAL, 'y'), (Action.ELSE_COND, 'x > y')])

traversal before [(True, "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))")]
previous (True, "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))")
prev expr Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))
after [(False, "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))")]


In [214]:
traversal_condition

[(False,
  "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))"),
 (True,
  "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[Name(id='y', ctx=Load())]))")]

In [215]:
branch_stack


[(<Action.IF_COND: 16>, [], (0, 3)),
 (<Action.ELSE_COND: 17>,
  [(False,
    "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))")],
  (3, 3))]

In [216]:
traversal_condition

[(False,
  "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))"),
 (True,
  "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[Name(id='y', ctx=Load())]))")]

In [217]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = True
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = True
updated = False
----------------------
name = temp
mode = Mode.LOCAL

In [218]:
 process_statement([(Action.EVAL,'x'), (Action.EVAL, 'y'), (Action.ASSIGN, 'y')]) 

In [219]:
print_state_stack(state_stack)

name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = False
updated = False
----------------------
name = temp
mode = Mode.LOCAL_RW
well_formed = (False, [])
scalar = True
lb = None
ub = None
invariant = False
evaluated = False
assigned = False
updated = False
----------------------
name = x
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = True
updated = False
----------------------
name = y
mode = Mode.IN_OUT_PARAM
well_formed = (True, [])
scalar = True
lb = -2147483648
ub = 2147483647
invariant = False
evaluated = True
assigned = True
updated = False
----------------------
name = temp
mode = Mode.LOCAL

In [220]:
process_statement([(Action.ELSE_BRANCH, '')])

merging  (0, 3)  and  (3, 3)


IndexError: list index out of range

In [193]:
branch_stack

[(<Action.IF_COND: 16>, [], (0, 3)),
 (<Action.ELSE_COND: 17>,
  [(False,
    "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))")],
  (3, 3)),
 (<Action.ELSE_BRANCH: 18>,
  [(False,
    "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=10))]))"),
   (False,
    "Expression(body=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[Name(id='y', ctx=Load())]))")],
  (6, 3))]